# ノートブック05: 決定木とアンサンブル 🌳

**学習目標**: 決定木、Random Forest、Gradient Boostingの基礎を完全にマスターする

⏱️ 推定時間: 120-150分 | 難易度: ★★★☆☆

## 📋 学習内容

1. 決定木の基礎（Gini不純度、情報利得）
2. Random Forest（バギング、特徴量重要度）
3. Gradient Boosting基礎
4. 実践例：Breast Cancer診断

**Note**: 高度なGBDT（LightGBM、XGBoost、CatBoost）はノートブック13-16で学びます